In [98]:
import numpy as np

In [120]:
# Define the nodal coordinates and element connectivity
coordinates = np.array([[0, 0], [1, 0], [0, 1], [1, 1]])
elements = np.array([[0, 1], [0, 2], [0, 3], [1, 3], [2, 3]])

In [121]:
# Define the material properties and cross-sectional area
E = 100e9
A = 0.01

In [122]:
def local_stiffness(A, E, L, theta):
    c = np.cos(theta)
    s = np.sin(theta)
    K_loc = (A * E / L) * np.array([[c**2,  c*s,   -c**2, -c*s], 
                                    [c*s,   s**2,  -c*s,  -s**2], 
                                    [-c**2, -c*s,  c**2,   c*s], 
                                    [-c*s,  -s**2, c*s,    s**2]])
    return K_loc

In [ ]:
# Define the global stiffness matrix
n_nodes = len(coordinates)
n_elements = len(elements)
K_global = np.zeros((2 * n_nodes, 2 * n_nodes))
for i in range(n_elements):
    element = elements[i]
    x1, y1 = coordinates[element[0]]
    x2, y2 = coordinates[element[1]]
    L = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    theta = np.arctan((y2 - y1) / (x2 - x1))
    K_e = local_stiffness(A, E, L, theta)
    dof = np.array([2 * element[0], 2 * element[0] + 1, 2 * element[1], 2 * element[1] + 1])
    K_global[np.ix_(dof, dof)] += K_e

In [115]:
# Apply boundary conditions
fixed_dofs = np.array([0, 1, 4])     # dofs  1, 2 are fixed
free_dofs = np.setdiff1d(np.arange(2 * n_nodes), fixed_dofs)  # the rest of the dofs are free
K_ff = K_global[np.ix_(free_dofs, free_dofs)]
K_fe = K_global[np.ix_(free_dofs, fixed_dofs)]
K_ef = K_global[np.ix_(fixed_dofs, free_dofs)]
F = np.zeros(2 * n_nodes)
F[7] = -1000  # apply a downward force at dof 7
F_f = F[free_dofs]
F_e = F[fixed_dofs]

In [116]:
# Solve the resulting system of linear equations
U_f = np.linalg.solve(K_ff, F_f)
U = np.zeros(2 * n_nodes)
U[free_dofs] = U_f

In [117]:
# Calculate reactions and element forces
R_e = K_ef @ U_f
R_global = np.zeros(2 * n_nodes)
R_global[fixed_dofs] += R_e
N_e = np.zeros(n_elements)
for i in range(n_elements):
    element = elements[i]
    dof = np.array([2 * element[0], 2 * element[0] + 1, 2 * element[1], 2 * element[1] + 1])
    U_e = U[dof]
    #K_e = K_global[np.ix_(dof, dof)]
    
    x1, y1 = coordinates[element[0]]
    x2, y2 = coordinates[element[1]]
    L = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    dx = (x2 - x1) / L
    dy = (y2 - y1) / L
    unit_vector = np.array([dx, dy])
    K_e = (A * E / L) * local_stiffness(dx, dy)
    F_e = K_e @ U_e + R_global[dof]
    F_e_proj = np.array([np.dot(F_e[0:2], unit_vector), np.dot(F_e[2:4], unit_vector)])
    N_e[i] = np.dot(F_e_proj, unit_vector)

In [118]:
# Print the results
print("Displacements:")
print(U.reshape(-1, 2))
print("Reactions:")
print(R_e)
print("Element forces:")
print(N_e)

Displacements:
[[ 0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -3.82842712e-06]
 [ 0.00000000e+00  0.00000000e+00]
 [ 1.00000000e-06 -3.82842712e-06]]
Reactions:
[ 1000.  1000. -1000.]
Element forces:
[ 1000.     0.  1000.     0. -2000.]
